# istftnet
[![Package badge]][github]
[![Open In Colab]][notebook]

Author: [tarepan]

[github]:https://github.com/tarepan/iSTFTNet
[notebook]:https://colab.research.google.com/github/tarepan/iSTFTNet/blob/main/istftnet.ipynb
[tarepan]:https://github.com/tarepan
[Package badge]:https://img.shields.io/badge/GitHub-istftnet-9cf.svg
[Open In Colab]:https://colab.research.google.com/assets/colab-badge.svg

## Colab Check
Check environments.

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}' # Google Colaboratory runnning time
!head -n 1 /proc/driver/nvidia/gpus/**/information                  # GPU type
!/usr/local/cuda/bin/nvcc --version | sed '4!d'                     # CUDA version
!python --version                                                   # Python version
!pip show torch | sed '2!d'                                         # PyTorch version

## Setup

Install the package from `tarepan/iSTFTNet` public repository

In [ ]:
# GoogleDrive
from google.colab import drive
drive.mount('/content/gdrive')

# Dedicated dependencies install
# !pip install "torch==2.0.1" "torchaudio==2.0.2" -q
!pip install "lightning==2.0.3" git+https://github.com/tarepan/lightlightning.git -q

# repository install
!pip uninstall istftnet -y -q
!pip install git+https://github.com/tarepan/iSTFTNet -q

## Training

In [ ]:
# Launch TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/gdrive/MyDrive/ML_results/istftnet

# Train
!python -m istftnet.main_train \
    train.ckpt_log.dir_root=/content/gdrive/MyDrive/ML_results/istftnet \
    train.ckpt_log.name_exp=2022 \
    train.ckpt_log.name_version=version_1 \
    data.adress_data_root=/content/gdrive/MyDrive/ML_data \
    # train.model.net.child.dim_o=10 \


## Inference

### From CLI

In [ ]:
!python -m istftnet.main_inference \
    --model-ckpt-path="/content/gdrive/MyDrive/ML_results/istftnet/test2/default/version_0/checkpoints/last.ckpt" \
    --i-path="./test.wav" \
    --o-path="./o.wav" \
    # --device="cpu" --device="cuda:0" \

### From Python

In [ ]:
import torch

from istftnet.model import Model


# Configs
model_ckpt_path = "<ckpt.pt>"
device = "cuda:0" # "cpu"

# Setup
model: Model = Model.load_from_checkpoint(checkpoint_path=model_ckpt_path).to(torch.device(device)) # type: ignore ; because of PyTorch Lightning
model.eval()

# Inference
with torch.inference_mode():
    # Raw data
    ####################################
    ## From sample
    raw = model.sample()
    ####################################
    ## From your file
    # from pathlib import Path
    # i_path = Path(f"<your_data>.xxx")
    # raw = model.load(i_path)
    ####################################
    ## From your upstream data
    # raw = <your_raw_data>
    ####################################

    batch = model.preprocess(raw, device)
    o_pred = model.predict_step(batch, batch_idx=0)

    # Tensor[Batch=1, ...] => Tensor[...] => NDArray[...]
    o_wave = o_pred[0].to('cpu').numpy()

# Output
print(o_wave)
##################################################
# Audio
##############################################
## To File
# import soundfile as sf
# sf.write(...)
##############################################
## To Notebook
# from IPython.display import Audio, display
# display(Audio(o_wave, rate=o_sr))
##############################################
##################################################


In [ ]:
# # Usage stat
# ## GPU
# !nvidia-smi -l 3
# ## CPU
# !vmstat 5
# !top